<a href="https://colab.research.google.com/github/Bilal-Waleed/PIAIC-Q2-P2/blob/main/PIAIC_Q2_Project_2_LangChain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 2: LangChain RAG Project**

In this Project, we will create a simple LangChain RAG Colab Notebook that uses the Google Gemini Flash model to answer user questions from the document provided. This example below is provided to help you get started assumes you have access to the Gemini API, Pinecone and a basic Python environment. However, you are required to develop and submit your project using Google Colab.

The project Github repo is: https://github.com/panaversity/learn-agentic-ai/blob/main/02_generative_ai_for_beginners/PROJECTS/02_rag

# **LangChain RAG with Google Gemini Flash and Pinecone**

### Installations

In [1]:
!pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 117.3 kB/s eta 0:00:00


### Configuring Pinecone Api

In [2]:
from IPython.display import Markdown, display
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('PINECONE-API-KEY')
pc = Pinecone(api_key=pinecone_api_key)

### **Creating Index**

In [3]:
index_name = "langchain-rag"  # change if desired

# List existing indexes
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# Check if the index already exists, if not, create it
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")

# Access the index
index = pc.Index(index_name)


Index 'langchain-rag' already exists.


### **Creating Embeddings Using Embedding model**

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

GEMINI_KEY = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001",google_api_key=GEMINI_KEY )

In [5]:
vector = embeddings.embed_query("Building a Rag project! ")

In [6]:
vector[:5]

[0.005886509083211422,
 -0.01920737698674202,
 -0.01310189813375473,
 -0.03790365159511566,
 -0.003551947884261608]

### **Creating Vector Store with Pinecone**

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [8]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I have chocolate chip pancake and scrambled eggs for breakfast",
    metadata={"source": "personal", "meal": "breakfast"}
)

document_2 = Document(
    page_content="LangChain is a framework for building applications with LLMs (Large Language Models), such as GPT. It provides tools to manage chains, agents, and memory for building more advanced AI applications.",
    metadata={"topic": "LangChain", "category": "AI Framework"}
)

document_3 = Document(
    page_content="Agentic AI refers to autonomous AI systems that are capable of decision-making, learning, and adaptation in real-world environments without needing constant human intervention.",
    metadata={"topic": "Agentic AI", "category": "Artificial Intelligence", "importance": "High"}
)

document_4 = Document(
    page_content="In the latest world news, a new tech company has developed an innovative AI that is able to solve real-world problems faster than previous models, pushing the boundaries of automation and efficiency.",
    metadata={"topic": "Tech News", "date": "2025-01-02", "company": "Innovative AI Company"}
)

document_5 = Document(
    page_content="The use of AI in healthcare is growing rapidly. Recent advancements in diagnostic AI tools are helping doctors identify diseases more accurately and faster, significantly improving patient outcomes.",
    metadata={"topic": "Healthcare AI", "category": "Medical Technology", "impact": "Positive"}
)

document_6 = Document(
    page_content="LangChain offers a wide range of tools to work with LLMs. This includes support for document search, question-answering systems, and memory management to make intelligent agents more effective in real-world tasks.",
    metadata={"topic": "LangChain", "category": "AI Tools", "use_case": "Advanced workflows"}
)

document_7 = Document(
    page_content="Agentic AI is becoming increasingly important in industries like autonomous vehicles, robotics, and smart cities, where real-time decision-making and adaptability are crucial for success.",
    metadata={"topic": "Agentic AI", "category": "Industry Applications", "industries": ["Autonomous Vehicles", "Robotics", "Smart Cities"]}
)

document_8 = Document(
    page_content="A new world record has been set for the fastest internet speed, with researchers breaking through previous limitations using advanced fiber-optic technology, promising faster and more efficient global communication.",
    metadata={"topic": "Tech News", "category": "Innovation", "record": "Fastest Internet Speed", "date": "2025-01-02"}
)

document_9 = Document(
    page_content="In a recent study, AI-powered chatbots have been shown to outperform human customer service agents in resolving technical issues, reducing wait times and improving customer satisfaction.",
    metadata={"topic": "AI in Customer Service", "category": "Business Technology", "impact": "High"}
)

document_10 = Document(
    page_content="LangChain continues to evolve with new integrations, including support for databases, APIs, and external data sources, enabling more complex and efficient workflows for AI applications.",
    metadata={"topic": "LangChain", "category": "Development", "features": ["Database Integration", "API Support", "External Data Sources"]}
)

documents = [
    document_1, document_2, document_3, document_4, document_5,
    document_6, document_7, document_8, document_9, document_10
]

In [9]:
len(  documents)

10

## Adding Documents and giving IDs

In [10]:
from uuid import uuid4
uuid4()

UUID('b9bfef99-8b19-4ad1-a393-6d3d45b82eb6')

In [11]:
uuids = [str(uuid4()) for i in range (len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['1d9cd3fa-2782-4ca5-8e62-fe19d9bf00f7',
 '6f5d9d5c-6473-402a-8e72-da9b7b5c4ad5',
 '40b4b222-7980-4af1-ad57-8a3e319770c2',
 '17562cb1-fe6f-4365-8796-0b4abbbf662a',
 'd3366639-68f5-4306-9113-74a602de3861',
 'f000ee3c-6994-4890-aeb5-1ee4e3ebda5d',
 'be1a73c0-4494-4b25-9580-aa1dbf5b2f98',
 '4558608b-91c2-4d08-bbfa-c113e04609f5',
 'bdc61fcc-fbed-48ba-b69c-46e11bfa2a30',
 'e3ffaae4-8f92-4c30-8f5d-2da7ed82a685']

### Performng Similarity search

In [18]:
vector_result = vector_store.similarity_search(
    "What is langchain", k=7,)
print(vector_result[0].page_content)

LangChain is a framework for building applications with LLMs (Large Language Models), such as GPT. It provides tools to manage chains, agents, and memory for building more advanced AI applications.


### Generating Contextual Answers with Google Generative AI

### **Using Langchain**

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate

In [20]:
def user_answer(question):

  vector_result = vector_store.similarity_search(question, k=5)

  llm = ChatGoogleGenerativeAI(
      api_key=GEMINI_KEY,
      model = "gemini-2.0-flash-exp",
      max_tokens= 100,
      temperature=0.7
  )

  prompt1= PromptTemplate(
      input_variables=["question"],
      template = "Using this data {vector_result} . Answer the following question: \n\n{question}"
  )
  chain1= prompt1 | llm

  final_answer = chain1.invoke({"vector_result": vector_result, "question": question})

  return final_answer

In [21]:
response = user_answer("what is langchain?")

In [22]:
display(Markdown(response.content))

Based on the provided documents, LangChain is a framework for building applications with Large Language Models (LLMs), such as GPT. It provides tools to manage chains, agents, and memory for building more advanced AI applications. Additionally, it continues to evolve with new integrations, including support for databases, APIs, and external data sources, enabling more complex and efficient workflows for AI applications.